In [ ]:
import numpy as np
import sklearn
import pandas as pd
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import ensemble,tree,linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier,GradientBoostingRegressor)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split,cross_val_score
# from sklearn.cross_validation import KFold
import tensorflow as tf
import warnings
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
py.init_notebook_mode(connected=True)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
warnings.filterwarnings('ignore')

pd.options.display.max_columns=99

In [ ]:
### Models

from sklearn import ensemble,tree,linear_model
from sklearn.metrics import r2_score, mean_squared_error

# Print R2 and RMSE scores
def get_score(prediction, labels):
    print('R2: {}'.format(r2_score(prediction, labels)))
    print('RMSE: {}'.format(np.sqrt(mean_squared_error(prediction,labels))))

In [ ]:
    
# Show scores for train and validation sets
def train_test(estimator, x_train, x_test, y_train, y_test):
    prediction_train = estimator.predict(x_train)
    print(estimator)
    get_score(prediction_train,y_train)
    
    prediction_test = estimator.predict(x_test)
    print('Test')
    get_score(prediction_test, y_test)

In [ ]:
# Elastic Net
def eNet(x_train,y_train,x_val,y_val):
    ENSTest = linear_model.ElasticNetCV(alphas=[0.0001,0.0005,0.001,0.01,0.1,1,10],l1_ratio=[.01, .1, .5, .9, .99], max_iter=5000).fit(x_train,y_train)
    train_test(ENSTest, x_train,x_train,y_train,y_train)

    # Average R2 score and standard deviation of 5-fold cross-validation
    scores = cross_val_score(ENSTest, x_train, y_train, cv=5)
    print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std()*2))

In [ ]:
# Gradient Boosting
def gBoost(x_train,y_train,x_val,y_val):
    GBest = ensemble.GradientBoostingRegressor(n_estimators=3000,learning_rate=0.05,max_depth=3,max_features='sqrt',
                                              min_samples_leaf=15,min_samples_split=10,loss='huber').fit(x_train,y_train)
    train_test(GBest,x_train,x_train,y_train,y_train)
    # Average R2 score and standard deviation of 5-fold cross-validation
    scores = cross_val_score(GBest, x_train, y_train, cv=5)
    print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std()*2))

    x_train.head()

In [ ]:
#### Neural networks

# # X_train = X_train_adas13[features].values
# y_train = y_train.values.reshape(-1,1)
# y_train.shape

def random_batch(x_train, y_train, batch_size):
    
    num = x_train.shape[0]

    # Create a random index.
    idx = np.random.choice(num,
                           size=batch_size,
                           replace=False)

    # Use the random index to select random images and labels.
    x_batch = x_train[idx]
    y_batch = y_train[idx]

    return x_batch, y_batch

In [ ]:
def NN(x_train,y_train,x_val,y_val):
    # Convert y_train shape to ?x1
    y_train = y_train.values.reshape(-1,1)

    n_input = x_train.shape[1]
    n_hidden1 = 128
    n_hidden2 = 512
    n_hidden3 = 1024
    n_output = 1
    learning_rate = 0.001
    epochs = 10000
    batch_size = 100
    REGULARIZATION_RATE = 0.0001

    X = tf.placeholder(tf.float32,[None,n_input])
    y_gt = tf.placeholder(tf.float32,[None,n_output])
    # regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)

    initializer = tf.contrib.layers.variance_scaling_initializer(factor=2.0, mode='FAN_IN', uniform=False)
    W1 = tf.Variable(initializer([n_input,n_hidden1]))
    # tf.add_to_collection('losses',regularizer(W1))
    b1 = tf.Variable(tf.constant(0.1,shape=[n_hidden1]))
    H1 = tf.nn.relu(tf.matmul(X,W1)+b1)

    W2 = tf.Variable(initializer([n_hidden1,n_hidden2]))
    # tf.add_to_collection('losses',regularizer(W2))
    b2 = tf.Variable(tf.constant(0.1,shape=[n_hidden2]))
    H2 = tf.nn.relu(tf.matmul(H1,W2)+b2)

    W3 = tf.Variable(initializer([n_hidden2,n_hidden3]))
    # tf.add_to_collection('losses',regularizer(W3))
    b3 = tf.Variable(tf.constant(0.1,shape=[n_hidden3]))
    H3 = tf.nn.relu(tf.matmul(H2,W3)+b3)

    W_out = tf.Variable(initializer([n_hidden3,n_output]))
    # tf.add_to_collection('losses',regularizer(W_out))
    b_out = tf.Variable(tf.constant(0.1,shape=[n_output]))
    y_pred = tf.matmul(H3,W_out)+b_out

    loss = tf.reduce_mean(tf.losses.mean_squared_error(labels=y_gt,predictions=y_pred)) 
    # + tf.add_n(tf.get_collection('losses'))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()

    for iter in range(epochs):

        sess.run(train_step, feed_dict={X:x_train, y_gt:y_train})
        if iter%1000 == 0:
            train_loss = sess.run(loss, feed_dict={X:x_train, y_gt:y_train})
            validation_loss = sess.run(loss, feed_dict={X:x_train, y_gt:y_train})
            print("Iter %d, training loss %f, validation loss %f" % (iter, train_loss, validation_loss))

In [ ]:
x_train_adas13=sess.run(y_pred,feed_dict={X:x_train})

y_train

X_test = X_test_adas13[features].values

adas13_pred = sess.run(y_pred,feed_dict={X:X_test})*100